In this notebook I'll show how to 
- maximize/minimize a univariate function in julia using the optim package
- how to define multivariate functions in julia and how to minimize those. 

(We do not cover general constraints but restrict ourselves to domain restrictions for now.)

# Minimizing a univariate function

We will find the minimum of the function $f(x)=(x-1)^2$. Admittedly, there is no need for using a computed to solve such a  simple problem but that makes it easier to show how to do it (and then you can use whatever complicated function you like).

We will use the Optim package. So, if you have not installed it yet, you need to install it now:

In [1]:
using Pkg
Pkg.add("Optim")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


Next I tell Julia to use Optim and define the function that I want to maximize.

In [3]:
using Optim

f(x)=(x-1)^2

f (generic function with 1 method)

and now we are ready for optimizing it: The command is "optimize" and its first argument is the function that you want  to optimize. Then you tell Optim a lower and an upper bound, i.e. you tell it an interval in which it whould look for the lowest function value. I choose -5 and 5 as boundaries which means that Optim will search for the x in the interval $[-5,5]$ that minimizes $f$. A you can see, Optim finds the minimizer 1 and the minimum 0.

In [4]:
optimize(f,-5,5)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-5.000000, 5.000000]
 * Minimizer: 1.000000e+00
 * Minimum: 0.000000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6

## ...but I want to maximize

If you want to maximize a function, say $g(x)=-(x-1)^2$, then you can just as well minimize the negative of the function; i.e. to solve $\max_x\, g(x)$ you can solve $\min_x\, -g(x)$. The minimizer is the maximizer in the original problem and the minimum is the negative of the maximum of the original problem. 

# Multivariate functions

First I have to tell you how to input multivariate functions into julia. I give several ways that all create the function 
$$g(z,y)=(1 - z)^2 + 100 * (y - z^2)^2$$

In [5]:
#option 1: math way
g(z,y)=(1 - z)^2 + 100 * (y - z^2)^2

#option 2: computer science way
function g(z,y)
    return (1 - z)^2 + 100 * (y - z^2)^2
end

g (generic function with 1 method)

There are further options: Instead of representing $g$ as a function of 2 variables $z$ and $y$ I could represent it as a function of one variable $x$ where this variable $x$ is a vector with two elements x[1] and x[2]. This is what I do next.

(Strangely enough this is the representation that we will need for the minimization problem below.)

In [6]:
h(x)=(1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2

function h(x)
    return (1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2
end

h (generic function with 1 method)

Now we can minimize this multivariate function, i.e. solve
$$\min_{y,z} (1 - z)^2 + 100 * (y - z^2)^2.$$
In Optim we represent this problem, however, as minimization over one variable $x$ which is a vector with two elements (what used to be z and y above) which we denote by x[1], the first element of the vector, and x[2], the second element of the vector. The minimization problem looks then like
$$\min_x  (1 - x[1])^2 + 100 * (x[2] - x[1]^2)^2.$$
Let's ask Optim to solve for the minimum. 

Unfortunately, some things change a bit compared to the univariate case above. Instead of telling optim an interval in which it should search, I am giving it a starting point for its algorithm, here [0.0,0.0]. A starting point is an approximate guess of the solution (the algorithm has an easier job if the starting value you give it is closer to the actual solution but for well behaved problems like minimizing a convex function the starting point does not matter too much). Note that you should write [0.0, 0.0] and not [0,0]. If you do the latter, Julia is not sure whether you want to consider integers, i.e. natural numbers, only (I think so at least...anyway: you get an error if you try [0,0]).

In [7]:
optimize(h,[0.0,0.0])

 * Status: success

 * Candidate solution
    Minimizer: [1.00e+00, 1.00e+00]
    Minimum:   3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    60
    f(x) calls:    118


## ...but I have domain restrictions

You might have a problem where variables are constraint, e.g. your variable represents a probability which can only be between 0 and 1. You can give these so called box-constraints to Optim also in the multivariate case. Say we need in the optimization problem above x[1] to be between 0 and 0.5 and x[2] between -2 and 1. We write this in the following way: After the lower bounds [-0.0,-2.0], we give the upper bounds [0.5,1.0] and then a starting value [0.2,0.0] and finally we tell Optim that we want to use a method that can handle box contraints. This method is called Fminbox and because FminBox is itself a function we have to write it as "Fminbox()".

In [14]:
optimize(h,[0.0,-2.0],[0.5,1.0],[0.2,0.0],Fminbox())

 * Status: success

 * Candidate solution
    Minimizer: [5.00e-01, 2.50e-01]
    Minimum:   2.500000e-01

 * Found with
    Algorithm:     Fminbox with L-BFGS
    Initial Point: [2.00e-01, 0.00e+00]

 * Convergence measures
    |x - x'|               = 2.78e-17 ≰ 0.0e+00
    |x - x'|/|x'|          = 4.97e-17 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.00e+00 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    7
    f(x) calls:    1241
    ∇f(x) calls:   1241
